# 训练 DeepSORT 多目标追踪算法

如果报错`CUDA out of memory.`则重启前面几个代码的`kernel`即可。

作者：同济子豪兄、张经伟 2022-4-25

## 进入 MMTracking 主目录

In [1]:
import os
os.chdir('mmtracking')
os.listdir()

['.git',
 '.circleci',
 '.dev_scripts',
 '.github',
 '.gitignore',
 '.pre-commit-config.yaml',
 '.readthedocs.yml',
 'CITATION.cff',
 'LICENSE',
 'MANIFEST.in',
 'README.md',
 'README_zh-CN.md',
 'configs',
 'demo',
 'docker',
 'docs',
 'mmtrack',
 'model-index.yml',
 'requirements.txt',
 'requirements',
 'resources',
 'setup.cfg',
 'setup.py',
 'tests',
 'tools',
 'mmtrack.egg-info',
 'checkpoints',
 'outputs',
 'data',
 '20220425141741',
 '20220425141741-plot',
 'test.jpg']

## 多目标追踪数据集准备

### 下载 MOT17 多目标追踪数据集

In [2]:
# 下载 MOT17 数据集压缩包
!wget https://download.openmmlab.com/mmtracking/data/MOT17_tiny.zip -P ./data

--2022-04-25 09:11:47--  https://download.openmmlab.com/mmtracking/data/MOT17_tiny.zip
Connecting to 172.16.0.13:5848... connected.
Proxy request sent, awaiting response... 200 OK
Length: 344566302 (329M) [application/zip]
Saving to: ‘./data/MOT17_tiny.zip’

MOT17_tiny.zip      100%[===================>] 328.60M  86.8MB/s    in 3.8s    

2022-04-25 09:11:51 (86.7 MB/s) - ‘./data/MOT17_tiny.zip’ saved [344566302/344566302]



In [3]:
# 解压
!rm -rf data/MOT17_tiny
!unzip ./data/MOT17_tiny.zip -d ./data

Archive:  ./data/MOT17_tiny.zip
   creating: ./data/MOT17_tiny/
   creating: ./data/MOT17_tiny/test/
   creating: ./data/MOT17_tiny/train/
   creating: ./data/MOT17_tiny/train/MOT17-02-FRCNN/
   creating: ./data/MOT17_tiny/train/MOT17-02-FRCNN/det/
  inflating: ./data/MOT17_tiny/train/MOT17-02-FRCNN/det/det.txt  
   creating: ./data/MOT17_tiny/train/MOT17-02-FRCNN/gt/
  inflating: ./data/MOT17_tiny/train/MOT17-02-FRCNN/gt/gt.txt  
  inflating: ./data/MOT17_tiny/train/MOT17-02-FRCNN/gt/gt_half-train.txt  
  inflating: ./data/MOT17_tiny/train/MOT17-02-FRCNN/gt/gt_half-val.txt  
   creating: ./data/MOT17_tiny/train/MOT17-02-FRCNN/img1/
  inflating: ./data/MOT17_tiny/train/MOT17-02-FRCNN/img1/000001.jpg  
  inflating: ./data/MOT17_tiny/train/MOT17-02-FRCNN/img1/000002.jpg  
  inflating: ./data/MOT17_tiny/train/MOT17-02-FRCNN/img1/000003.jpg  
  inflating: ./data/MOT17_tiny/train/MOT17-02-FRCNN/img1/000004.jpg  
  inflating: ./data/MOT17_tiny/train/MOT17-02-FRCNN/img1/000005.jpg  
  inflati

### 将 txt 格式的标注文件，转换为 coco 格式的 json 文件

In [4]:
!python ./tools/convert_datasets/mot/mot2coco.py -i ./data/MOT17_tiny/ -o ./data/MOT17_tiny/annotations --split-train --convert-det

Converting train set to COCO format
100%|█████████████████████████████████████████████| 2/2 [00:00<00:00,  2.29it/s]
train has 145 instances.
Done! Saved as ./data/MOT17_tiny/annotations/train_cocoformat.json and ./data/MOT17_tiny/annotations/train_detections.pkl
Converting test set to COCO format
0it [00:00, ?it/s]
test has 0 instances.
Done! Saved as ./data/MOT17_tiny/annotations/test_cocoformat.json and ./data/MOT17_tiny/annotations/test_detections.pkl
Converting half-train set to COCO format
100%|█████████████████████████████████████████████| 2/2 [00:02<00:00,  1.15s/it]
half-train has 104 instances.
Done! Saved as ./data/MOT17_tiny/annotations/half-train_cocoformat.json and ./data/MOT17_tiny/annotations/half-train_detections.pkl
Converting half-val set to COCO format
100%|█████████████████████████████████████████████| 2/2 [00:02<00:00,  1.12s/it]
half-val has 122 instances.
Done! Saved as ./data/MOT17_tiny/annotations/half-val_cocoformat.json and ./data/MOT17_tiny/annotations/half

### 从数据集中把行人的图块裁剪出来，用于训练 ReID 重识别模型

In [5]:
# 删除已有的 reid 目录（如有）
!rm -rf ./data/MOT17_tiny/reid

In [6]:
# 大概需要 20 分钟
!python ./tools/convert_datasets/mot/mot2reid.py -i ./data/MOT17_tiny/ -o ./data/MOT17_tiny/reid --val-split 0.9 --vis-threshold 0.8

100%|████████████████████████████████████████████| 2/2 [20:51<00:00, 625.77s/it]


## 分别训练 DeepSORT 模型中的不同模块

### 配置 DeepSORT 中的目标检测算法模块

In [7]:
import mmcv
from mmdet.apis import set_random_seed

cfg = mmcv.Config.fromfile('./configs/det/faster-rcnn_r50_fpn_4e_mot17-half.py')
cfg.data_root = 'data/MOT17_tiny/'
cfg.data.test.ann_file = cfg.data.test.ann_file.replace('data/MOT17/','data/MOT17_tiny/')
cfg.data.train.ann_file = cfg.data.train.ann_file.replace('data/MOT17/','data/MOT17_tiny/')
cfg.data.val.ann_file = cfg.data.val.ann_file.replace('data/MOT17/','data/MOT17_tiny/')

cfg.data.test.img_prefix = cfg.data.test.img_prefix.replace('data/MOT17/','data/MOT17_tiny/')
cfg.data.train.img_prefix = cfg.data.train.img_prefix.replace('data/MOT17/','data/MOT17_tiny/')
cfg.data.val.img_prefix = cfg.data.val.img_prefix.replace('data/MOT17/','data/MOT17_tiny/')

cfg.work_dir = './tutorial_exps/detector'
cfg.seed = 0
set_random_seed(0, deterministic=False)
cfg.gpu_ids = range(1)

In [8]:
print(cfg.pretty_text)

model = dict(
    detector=dict(
        type='FasterRCNN',
        backbone=dict(
            type='ResNet',
            depth=50,
            num_stages=4,
            out_indices=(0, 1, 2, 3),
            frozen_stages=1,
            norm_cfg=dict(type='BN', requires_grad=True),
            norm_eval=True,
            style='pytorch',
            init_cfg=dict(
                type='Pretrained', checkpoint='torchvision://resnet50')),
        neck=dict(
            type='FPN',
            in_channels=[256, 512, 1024, 2048],
            out_channels=256,
            num_outs=5),
        rpn_head=dict(
            type='RPNHead',
            in_channels=256,
            feat_channels=256,
            anchor_generator=dict(
                type='AnchorGenerator',
                scales=[8],
                ratios=[0.5, 1.0, 2.0],
                strides=[4, 8, 16, 32, 64]),
            bbox_coder=dict(
                type='DeltaXYWHBBoxCoder',
                target_means=[0.0, 0.0, 0.

In [13]:
import os.path as osp

from mmtrack.datasets import build_dataset
from mmdet.apis import train_detector as train_model
from mmdet.models import build_detector as build_model

mmcv.mkdir_or_exist(osp.abspath(cfg.work_dir))
model = build_model(cfg.model.detector)

datasets = [build_dataset(cfg.data.train)]
model.CLASSES = datasets[0].CLASSES

loading annotations into memory...
Done (t=0.26s)
creating index...
index created!


### 训练 DeepSORT 中的目标检测算法模块

In [12]:
# model.init_weights()

In [14]:
train_model(model, datasets, cfg)

/environment/miniconda3/lib/python3.7/site-packages/mmdet/apis/train.py:135: UserWarning: config is now expected to have a `runner` section, please set `runner` in your config.
  'please set `runner` in your config.', UserWarning)
2022-04-25 09:37:10,410 - mmdet - INFO - Start running, host: featurize@featurize, work_dir: /home/featurize/work/MMTracking教程/0421/mmtracking/tutorial_exps/detector
2022-04-25 09:37:10,411 - mmdet - INFO - Hooks will be executed in the following order:
before_run:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) CheckpointHook                     
(VERY_LOW    ) TextLoggerHook                     
 -------------------- 
before_train_epoch:
(VERY_HIGH   ) StepLrUpdaterHook                  
(LOW         ) IterTimerHook                      
(VERY_LOW    ) TextLoggerHook                     
 -------------------- 
before_train_iter:
(VERY_HIGH   ) StepLrUpdaterHook                  
(LOW         ) IterTimerHook                      
 ---------

### 配置 DeepSORT 中的 ReID 重识别算法模块

ReID 重识别算法本质上是一个多分类图像分类模型，输入行人的 patch 图，提取图中特征，输出 ID 号。

用来从图像特征中判断当前行人是否是新人。

In [16]:
import mmcv
from mmdet.apis import set_random_seed

cfg = mmcv.Config.fromfile('./configs/reid/resnet50_b32x8_MOT17.py')
cfg.data_root = 'data/MOT17_tiny/'
cfg.data.test.ann_file = cfg.data.test.ann_file.replace('data/MOT17/','data/MOT17_tiny/')
cfg.data.train.ann_file = 'data/MOT17_tiny/reid/meta/train_9.txt'
cfg.data.val.ann_file = cfg.data.val.ann_file.replace('data/MOT17/','data/MOT17_tiny/')

cfg.data.test.data_prefix = cfg.data.test.data_prefix.replace('data/MOT17/','data/MOT17_tiny/')
cfg.data.train.data_prefix = cfg.data.train.data_prefix.replace('data/MOT17/','data/MOT17_tiny/')
cfg.data.val.data_prefix = cfg.data.val.data_prefix.replace('data/MOT17/','data/MOT17_tiny/')

# 学习率策略
cfg.lr_config = dict(
    policy='step',
    warmup='linear',
    warmup_iters=200,
    warmup_ratio=1.0 / 200,
    step=[1])

cfg.total_epochs = 2

cfg.work_dir = './tutorial_exps/reid'
cfg.seed = 0
set_random_seed(0, deterministic=False)
cfg.gpu_ids = range(1)

In [17]:
print(cfg.pretty_text)

dataset_type = 'ReIDDataset'
img_norm_cfg = dict(
    mean=[123.675, 116.28, 103.53], std=[58.395, 57.12, 57.375], to_rgb=True)
train_pipeline = [
    dict(type='LoadMultiImagesFromFile', to_float32=True),
    dict(
        type='SeqResize',
        img_scale=(128, 256),
        share_params=False,
        keep_ratio=False,
        bbox_clip_border=False,
        override=False),
    dict(
        type='SeqRandomFlip',
        share_params=False,
        flip_ratio=0.5,
        direction='horizontal'),
    dict(
        type='SeqNormalize',
        mean=[123.675, 116.28, 103.53],
        std=[58.395, 57.12, 57.375],
        to_rgb=True),
    dict(type='VideoCollect', keys=['img', 'gt_label']),
    dict(type='ReIDFormatBundle')
]
test_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='Resize', img_scale=(128, 256), keep_ratio=False),
    dict(
        type='Normalize',
        mean=[123.675, 116.28, 103.53],
        std=[58.395, 57.12, 57.375],
        to_rgb=True),
    dic

In [18]:
from mmtrack.datasets import build_dataset
from mmdet.apis import train_detector as train_model
from mmtrack.models import build_reid as build_model


model = build_model(cfg.model.reid)
model.init_weights()
datasets = [build_dataset(cfg.data.train)]
model.CLASSES = datasets[0].CLASSES

2022-04-25 09:52:01,538 - mmcv - INFO - initialize BaseReID with init_cfg {'type': 'Pretrained', 'checkpoint': 'https://download.openmmlab.com/mmclassification/v0/resnet/resnet50_batch256_imagenet_20200708-cfb998bf.pth'}
2022-04-25 09:52:01,540 - mmcv - INFO - load model from: https://download.openmmlab.com/mmclassification/v0/resnet/resnet50_batch256_imagenet_20200708-cfb998bf.pth
2022-04-25 09:52:01,541 - mmcv - INFO - load checkpoint from http path: https://download.openmmlab.com/mmclassification/v0/resnet/resnet50_batch256_imagenet_20200708-cfb998bf.pth
Downloading: "https://download.openmmlab.com/mmclassification/v0/resnet/resnet50_batch256_imagenet_20200708-cfb998bf.pth" to /home/featurize/.cache/torch/hub/checkpoints/resnet50_batch256_imagenet_20200708-cfb998bf.pth


  0%|          | 0.00/97.7M [00:00<?, ?B/s]

2022-04-25 09:52:03,215 - mmcv - WARNING - The model and loaded state dict do not match exactly

unexpected key in source state_dict: head.fc.weight, head.fc.bias

missing keys in source state_dict: head.fcs.0.fc.weight, head.fcs.0.fc.bias, head.fcs.0.bn.weight, head.fcs.0.bn.bias, head.fcs.0.bn.running_mean, head.fcs.0.bn.running_var, head.fc_out.weight, head.fc_out.bias, head.bn.weight, head.bn.bias, head.bn.running_mean, head.bn.running_var, head.classifier.weight, head.classifier.bias



### 训练 DeepSORT 中的 ReID 重识别算法模块

In [19]:
train_model(model, datasets, cfg)

2022-04-25 09:52:06,400 - mmdet - INFO - Start running, host: featurize@featurize, work_dir: /home/featurize/work/MMTracking教程/0421/mmtracking/tutorial_exps/reid
2022-04-25 09:52:06,402 - mmdet - INFO - Hooks will be executed in the following order:
before_run:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) CheckpointHook                     
(VERY_LOW    ) TextLoggerHook                     
 -------------------- 
before_train_epoch:
(VERY_HIGH   ) StepLrUpdaterHook                  
(LOW         ) IterTimerHook                      
(VERY_LOW    ) TextLoggerHook                     
 -------------------- 
before_train_iter:
(VERY_HIGH   ) StepLrUpdaterHook                  
(LOW         ) IterTimerHook                      
 -------------------- 
after_train_iter:
(ABOVE_NORMAL) OptimizerHook                      
(NORMAL      ) CheckpointHook                     
(LOW         ) IterTimerHook                      
(VERY_LOW    ) TextLoggerHook                     


## 测试训练得到的 DeepSORT 多目标追踪算法

### 生成 config 配置文件

In [20]:
import mmcv
from mmdet.apis import set_random_seed

cfg = mmcv.Config.fromfile('./configs/mot/deepsort/deepsort_faster-rcnn_fpn_4e_mot17-private-half.py')
cfg.data_root = 'data/MOT17_tiny/'
cfg.data.test.ann_file = cfg.data.test.ann_file.replace('data/MOT17/','data/MOT17_tiny/')
cfg.data.train.ann_file = cfg.data.test.ann_file.replace('data/MOT17/','data/MOT17_tiny/')
cfg.data.val.ann_file = cfg.data.val.ann_file.replace('data/MOT17/','data/MOT17_tiny/')

cfg.data.test.img_prefix = cfg.data.test.img_prefix.replace('data/MOT17/','data/MOT17_tiny/')
cfg.data.train.img_prefix = cfg.data.train.img_prefix.replace('data/MOT17/','data/MOT17_tiny/')
cfg.data.val.img_prefix = cfg.data.val.img_prefix.replace('data/MOT17/','data/MOT17_tiny/')

# 指定训练好的目标检测模块和 ReID 重识别模块
cfg.model.detector.init_cfg.checkpoint = './tutorial_exps/detector/epoch_4.pth'
cfg.model.reid.init_cfg.checkpoint = './tutorial_exps/reid/epoch_2.pth'

cfg.work_dir = './tutorial_exps'
cfg.seed = 0
set_random_seed(0, deterministic=False)
cfg.gpu_ids = range(1)
cfg.data.test.test_mode = True

In [21]:
print(cfg.pretty_text)

model = dict(
    detector=dict(
        type='FasterRCNN',
        backbone=dict(
            type='ResNet',
            depth=50,
            num_stages=4,
            out_indices=(0, 1, 2, 3),
            frozen_stages=1,
            norm_cfg=dict(type='BN', requires_grad=True),
            norm_eval=True,
            style='pytorch',
            init_cfg=dict(
                type='Pretrained', checkpoint='torchvision://resnet50')),
        neck=dict(
            type='FPN',
            in_channels=[256, 512, 1024, 2048],
            out_channels=256,
            num_outs=5),
        rpn_head=dict(
            type='RPNHead',
            in_channels=256,
            feat_channels=256,
            anchor_generator=dict(
                type='AnchorGenerator',
                scales=[8],
                ratios=[0.5, 1.0, 2.0],
                strides=[4, 8, 16, 32, 64]),
            bbox_coder=dict(
                type='DeltaXYWHBBoxCoder',
                target_means=[0.0, 0.0, 0.

### 保存 config 配置文件

In [ ]:
!wget https://zihao-openmmlab.obs.cn-east-3.myhuaweicloud.com/20220418-mmtracking/deepsort_faster-rcnn_fpn_4e_mot17-new.py -O configs/mot/deepsort/deepsort_faster-rcnn_fpn_4e_mot17-new.py

### 构建模型

In [24]:
from mmtrack.datasets import build_dataloader
from mmtrack.models import build_model
from mmcv.parallel import MMDataParallel
from mmtrack.apis import single_gpu_test
from mmtrack.datasets import build_dataset

dataset = build_dataset(cfg.data.test)
data_loader = build_dataloader(
    dataset,
    samples_per_gpu=1,
    workers_per_gpu=cfg.data.workers_per_gpu,
    dist=False,
    shuffle=False)

# 构建模型，载入 checkpoint 权重文件
model = build_model(cfg.model)
model.init_weights()

model = MMDataParallel(model, device_ids=cfg.gpu_ids)

loading annotations into memory...
Done (t=0.17s)
creating index...
index created!


2022-04-25 10:08:12,534 - mmcv - INFO - initialize FasterRCNN with init_cfg {'type': 'Pretrained', 'checkpoint': './tutorial_exps/detector/epoch_4.pth'}
2022-04-25 10:08:12,536 - mmcv - INFO - load model from: ./tutorial_exps/detector/epoch_4.pth
2022-04-25 10:08:12,536 - mmcv - INFO - load checkpoint from local path: ./tutorial_exps/detector/epoch_4.pth
2022-04-25 10:08:12,856 - mmcv - INFO - initialize BaseReID with init_cfg {'type': 'Pretrained', 'checkpoint': './tutorial_exps/reid/epoch_2.pth'}
2022-04-25 10:08:12,857 - mmcv - INFO - load model from: ./tutorial_exps/reid/epoch_2.pth
2022-04-25 10:08:12,858 - mmcv - INFO - load checkpoint from local path: ./tutorial_exps/reid/epoch_2.pth
2022-04-25 10:08:13,054 - mmcv - INFO - 
detector.backbone.conv1.weight - torch.Size([64, 3, 7, 7]): 
PretrainedInit: load from ./tutorial_exps/detector/epoch_4.pth 
 
2022-04-25 10:08:13,055 - mmcv - INFO - 
detector.backbone.bn1.weight - torch.Size([64]): 
The value is the same before and after ca

### 在测试集上预测

In [25]:
outputs = single_gpu_test(model, data_loader)

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 823/823, 4.8 task/s, elapsed: 170s, ETA:     0s

### 在测试集上评估

In [27]:
eval_kwargs = cfg.get('evaluation', {}).copy()
# hard-code way to remove EvalHook args
eval_hook_args = [
    'interval', 'tmpdir', 'start', 'gpu_collect', 'save_best',
    'rule', 'by_epoch'
]
for key in eval_hook_args:
    eval_kwargs.pop(key, None)
eval_kwargs.update(dict(metric=['track']))
metric = dataset.evaluate(outputs, **eval_kwargs)
print(metric)

Evaluate CLEAR MOT results.

Eval Config:
USE_PARALLEL         : False                         
NUM_PARALLEL_CORES   : 8                             
BREAK_ON_ERROR       : True                          
RETURN_ON_ERROR      : False                         
LOG_ON_ERROR         : /environment/miniconda3/lib/python3.7/site-packages/error_log.txt
PRINT_RESULTS        : True                          
PRINT_ONLY_COMBINED  : False                         
PRINT_CONFIG         : True                          
TIME_PROGRESS        : True                          
DISPLAY_LESS_PROGRESS : True                          
OUTPUT_SUMMARY       : True                          
OUTPUT_EMPTY_CLASSES : True                          
OUTPUT_DETAILED      : True                          
PLOT_CURVES          : True                          

MotChallenge2DBox Config:
GT_FOLDER            : data/MOT17_tiny/train         
TRACKERS_FOLDER      : /tmp/tmpxnoiudbt              
OUTPUT_FOLDER        : None    

<Figure size 432x288 with 0 Axes>

### 对新视频预测

In [28]:
# 命令行方式实现
# Deepsort算法仅需指定 config 文件，不需指定 checkpoint 文件
!python demo/demo_mot_vis.py \
        configs/mot/deepsort/deepsort_faster-rcnn_fpn_4e_mot17-new.py \
        --input data/mot_people_short.mp4 \
        --output outputs/I1_MOT_people_short.mp4

2022-04-25 10:34:31,364 - mmtrack - INFO - initialize FasterRCNN with init_cfg {'type': 'Pretrained', 'checkpoint': './tutorial_exps/detector/epoch_4.pth'}
2022-04-25 10:34:31,365 - mmcv - INFO - load model from: ./tutorial_exps/detector/epoch_4.pth
2022-04-25 10:34:31,365 - mmcv - INFO - load checkpoint from local path: ./tutorial_exps/detector/epoch_4.pth
2022-04-25 10:34:34,905 - mmtrack - INFO - initialize BaseReID with init_cfg {'type': 'Pretrained', 'checkpoint': './tutorial_exps/reid/epoch_2.pth'}
2022-04-25 10:34:34,905 - mmcv - INFO - load model from: ./tutorial_exps/reid/epoch_2.pth
2022-04-25 10:34:34,905 - mmcv - INFO - load checkpoint from local path: ./tutorial_exps/reid/epoch_2.pth
[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 99/99, 4.9 task/s, elapsed: 20s, ETA:     0smaking the output video at outputs/I1_MOT_people_short.mp4 with a FPS of 24
[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 99/99, 26.6 task/s, elapsed: 4s, ETA:     0s


In [29]:
# Python API 方式实现
import mmcv
import tempfile
from mmtrack.apis import inference_mot, init_model

# 输入输出视频路径
input_video = 'data/mot_people_short.mp4'
output = 'outputs/I2_MOT_people_short.mp4'

# 指定 config 配置文件 和 模型权重文件，创建模型
mot_config = './configs/mot/deepsort/deepsort_faster-rcnn_fpn_4e_mot17-new.py'
# 初始化模型
mot_model = init_model(mot_config, device='cuda:0')

# 读入待预测视频
imgs = mmcv.VideoReader(input_video)
prog_bar = mmcv.ProgressBar(len(imgs))
out_dir = tempfile.TemporaryDirectory()
out_path = out_dir.name
# 逐帧输入模型预测
for i, img in enumerate(imgs):
    result = inference_mot(mot_model, img, frame_id=i)
    
    mot_model.show_result(
            img,
            result,
            show=False,
            wait_time=int(1000. / imgs.fps),
            out_file=f'{out_path}/{i:06d}.jpg')
    prog_bar.update()

print(f'\n making the output video at {output} with a FPS of {imgs.fps}')
mmcv.frames2video(out_path, output, fps=imgs.fps, fourcc='mp4v')
out_dir.cleanup()

2022-04-25 10:35:02,141 - mmcv - INFO - initialize FasterRCNN with init_cfg {'type': 'Pretrained', 'checkpoint': './tutorial_exps/detector/epoch_4.pth'}
2022-04-25 10:35:02,142 - mmcv - INFO - load model from: ./tutorial_exps/detector/epoch_4.pth
2022-04-25 10:35:02,142 - mmcv - INFO - load checkpoint from local path: ./tutorial_exps/detector/epoch_4.pth
2022-04-25 10:35:02,399 - mmcv - INFO - initialize BaseReID with init_cfg {'type': 'Pretrained', 'checkpoint': './tutorial_exps/reid/epoch_2.pth'}
2022-04-25 10:35:02,400 - mmcv - INFO - load model from: ./tutorial_exps/reid/epoch_2.pth
2022-04-25 10:35:02,401 - mmcv - INFO - load checkpoint from local path: ./tutorial_exps/reid/epoch_2.pth
2022-04-25 10:35:02,545 - mmcv - INFO - 
detector.backbone.conv1.weight - torch.Size([64, 3, 7, 7]): 
PretrainedInit: load from ./tutorial_exps/detector/epoch_4.pth 
 
2022-04-25 10:35:02,546 - mmcv - INFO - 
detector.backbone.bn1.weight - torch.Size([64]): 
The value is the same before and after ca

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 99/99, 4.7 task/s, elapsed: 21s, ETA:     0s
 making the output video at outputs/I2_MOT_people_short.mp4 with a FPS of 24.652929091701427
[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 99/99, 25.6 task/s, elapsed: 4s, ETA:     0s
